In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, ParameterGrid

import matplotlib.pyplot as plt
import utils
import pandas as pd

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
MAIN_PATH = "/mnt/homeGPU/fcastro/MINDAT_AVAN/fran/"
#MAIN_PATH = "/home/franblue/PROYECTOS/lulc/"

#data_dir = "../reduced_data/"
data_dir = MAIN_PATH + "data/"
batch_size = 32
img_height = 224
img_width = 224

In [4]:
train_data, train_labels, test_data, test_labels, test_names = utils.load_data(data_dir, norm=False)

In [5]:
idx = np.logical_or(train_labels==20, train_labels==21)
train_data = train_data[idx]
train_data=train_data/127.5-1.0
train_labels = train_labels[idx]

idx = np.logical_or(test_labels==20, test_labels==21)
test_data = test_data[idx]
test_data=test_data/127.5-1.0
test_labels = test_labels[idx]

In [6]:
train_labels_oh = train_labels.copy()
train_labels_oh[train_labels_oh==20] = 0
train_labels_oh[train_labels_oh==21] = 1
train_labels_oh = tf.one_hot(train_labels_oh, 2).numpy()

test_labels_oh = test_labels.copy()
test_labels_oh[test_labels_oh==20] = 0
test_labels_oh[test_labels_oh==21] = 1
test_labels_oh = tf.one_hot(test_labels_oh, 2).numpy()

In [7]:
fl_train_data = train_data.reshape(train_data.shape[0], -1)
fl_test_data = test_data.reshape(test_data.shape[0], -1)
pca = PCA()
pca = pca.fit(fl_train_data)
fl_train_data_trans = pca.transform(fl_train_data)
fl_test_data_trans = pca.transform(fl_test_data)

In [29]:
inputs = tf.keras.layers.Input(shape=(fl_train_data_trans.shape[1]))
x = tf.keras.layers.Dense(128)(inputs)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dense(16)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

In [30]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.RMSprop(0.00001)
metrics = [tf.keras.metrics.categorical_accuracy]
loss = tf.keras.losses.CategoricalCrossentropy()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=10, verbose=0, mode='auto', restore_best_weights=True)
callbacks = [early_stopping]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [31]:
hist = model.fit(fl_train_data_trans, train_labels_oh, epochs=5, batch_size=16, validation_split=0.2, callbacks=callbacks)
#hist = model.fit(fl_train_data_trans, train_labels_oh, epochs=100, batch_size=32, validation_split=0.2, callbacks=callbacks)

Epoch 1/5
36/36 [==============================] - 1s 9ms/step - loss: 0.7991 - categorical_accuracy: 0.4788 - val_loss: 0.3000 - val_categorical_accuracy: 0.9225
Epoch 2/5
36/36 [==============================] - 0s 5ms/step - loss: 0.7626 - categorical_accuracy: 0.4859 - val_loss: 0.3756 - val_categorical_accuracy: 0.8662
Epoch 3/5
36/36 [==============================] - 0s 5ms/step - loss: 0.7491 - categorical_accuracy: 0.5212 - val_loss: 0.4341 - val_categorical_accuracy: 0.8451
Epoch 4/5
36/36 [==============================] - 0s 5ms/step - loss: 0.7209 - categorical_accuracy: 0.5300 - val_loss: 0.4780 - val_categorical_accuracy: 0.7958
Epoch 5/5
36/36 [==============================] - 0s 5ms/step - loss: 0.7174 - categorical_accuracy: 0.5477 - val_loss: 0.5083 - val_categorical_accuracy: 0.7465


In [32]:
preds = model.predict(fl_test_data_trans)
accuracy_score(np.argmax(test_labels_oh, axis=-1), np.argmax(preds, axis=-1))

0.36893203883495146